# 2.3 Individual Model 3-lawler

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import optim, nn
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Device: ",device,"Device Count: ", torch.cuda.device_count(), "Device Name: ",torch.cuda.get_device_name()  )
    print("Torch version:", torch.__version__)
device

Device:  cuda Device Count:  2 Device Name:  Tesla V100-PCIE-16GB
Torch version: 1.7.1


device(type='cuda')

In [2]:
import sys

### imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow.compat.v1 as tf # maintain compatibility with TensorFlow 2.2.0

import keras
# from tensorflow.compat.v1.keras import backend as K # changed for compatibility with TensorFlow 2.2.0
import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.utils import np_utils
from keras.regularizers import l1
from keras.models import load_model, save_model

from tensorflow.compat.v1.keras import backend as K
import numpy as np

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


random_seed  = 1000
batch_size = 128

In [3]:
# Load my models and functions
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices
from utils import *
from model import *


In [4]:
EPOCHS = 30
winmin = 3 
stridesec = 15

print_settings(winmin,stridesec, EPOCHS)

*****************************************************************

Execution Started at 12/20/2020, 20:03:40
WindowLength: 3.00 min (2700 datum)	Slide: 15 (225 datum)	Epochs:30



In [5]:
person = "lawler"
meal_data = Person_MealsDataset(person_name= person, file_name = "all_files_list", winmin = winmin,stridesec = stridesec)
samples,labels =  meal_data.data_indices, meal_data.labels
# split train set and test set
train_indices, test_indices = split_train_test_indices(X= [i for i in range(len(labels))],
                                                        y = labels, test_size = 0.2,
                                                       random_seed = random_seed)
# balance train set
trainset_labels = labels[train_indices]
train_indices_balanced = balance_data_indices(trainset_labels,data_indices= train_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)


Loading Dataset ...
Loading File:  ../data/IndividualData/lawler-data/10.10/10.10.shm
Loading File:  ../data/IndividualData/lawler-data/10.12/10.12.shm
Loading File:  ../data/IndividualData/lawler-data/10.14/10.14.shm
Loading File:  ../data/IndividualData/lawler-data/10.16/10.16.shm
Loading File:  ../data/IndividualData/lawler-data/10.19/10.19.shm
Loading File:  ../data/IndividualData/lawler-data/10.20/afternoon_2hr33min/10.20.shm
Loading File:  ../data/IndividualData/lawler-data/10.20/evening_2hr_20min/10.20.shm
Loading File:  ../data/IndividualData/lawler-data/10.21/10.21.shm
Loading File:  ../data/IndividualData/lawler-data/10.26/10.26.shm
Loading File:  ../data/IndividualData/lawler-data/10.28/10.28.shm
Loading File:  ../data/IndividualData/lawler-data/10.31/10.31.shm
Loading File:  ../data/IndividualData/lawler-data/10.31/evening-2hr_goodDinnerTemplate_CFAmeal/10.31.shm
Loading File:  ../data/IndividualData/lawler-data/10.4/10.4.shm
Loading File:  ../data/IndividualData/lawler-dat

In [6]:
train_set_balanced = torch.utils.data.Subset(meal_data, train_indices_balanced)
test_set = torch.utils.data.Subset(meal_data, test_indices)

train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set ,batch_size=batch_size, shuffle=True,num_workers=2)

print("Data Loader Created")

Data Loader Created


In [7]:
torch.manual_seed(random_seed)

# split validation set
balanced_trainset_labels = labels[train_indices_balanced]
train_indices, valid_indices = split_train_test_indices(X= train_indices_balanced,
                                                        y = balanced_trainset_labels, test_size = 0.2,
                                                       random_seed = random_seed)

valid_set_balanced = torch.utils.data.Subset(meal_data, valid_indices)
valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)

# train model
input_shape = (train_set_balanced[0][0].shape[0], train_set_balanced[0][0].shape[1])
model_1 = Discriminator_ResNet( ngpu=1, input_shape =input_shape , out_fea = 1)
_ = model_1(torch.rand((1, input_shape[0],input_shape[1])))
model_1.to(device)
criterion_1 = nn.BCEWithLogitsLoss()
optimizer_1 = optim.Adam(model_1.parameters(),lr=0.01,  weight_decay=0.1)
lrscheduler_1 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_1, mode='min',patience= 2, factor = 0.1,threshold=0.01)
dataloader = (train_loader, valid_loader)
model_1, best_model_1,val_score_1,loss_ls_1, train_acc_ls_1, valid_acc_ls_1 = train_model(model_1,dataloader, optimizer_1, 
                                                                    criterion_1, lrscheduler_1, device= device,
                                                                    n_epochs=30, patience = 5, l1_enabled=False,
                                                                    checkpoint_name ="../models/lawler_models/checkpoint_model_resnet.pt" )

Train set size: 2300, with 1150 positive samples and 1150 negative samples
Test set size: 576, with 288 positive samples and 288 negative samples
Training set batch amounts: 23
Test set : 5
Start Training..
Epoch: 0,  Epoch_Loss: 1.7469, Train Acc: 51.1822 %, Train Recall: 0.4854, Validation Acc:  54.8611 %,  Validation Recall: 0.2049  
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 0.7495, Train Acc: 55.8762 %, Train Recall: 0.5918, Validation Acc:  55.3819 %,  Validation Recall: 0.2882  
Checkpoint Saved


Epoch: 2,  Epoch_Loss: 0.6452, Train Acc: 60.5355 %, Train Recall: 0.6022, Validation Acc:  60.2431 %,  Validation Recall: 0.5104  
Checkpoint Saved


Epoch: 3,  Epoch_Loss: 0.6069, Train Acc: 66.3769 %, Train Recall: 0.6961, Validation Acc:  64.2361 %,  Validation Recall: 0.9201  
Checkpoint Saved


Epoch: 4,  Epoch_Loss: 0.5705, Train Acc: 71.5925 %, Train Recall: 0.7629, Validation Acc:  70.1389 %,  Validation Recall: 0.7882  
Checkpoint Saved


Epoch: 5,  Epoch_Loss: 0.5490, Train A

In [8]:
best_model_1.eval()
acc, recall = eval_model(best_model_1, test_loader,device)
print("Test Accuracy:", acc)

Test Accuracy: 68.82367695008779


In [8]:
# Test to check if random generated indices can be reproducted
# ls = balance_data_indices(traininglabels,mode="under", shuffle=True,random_state = random_seed)
# ls ==train_shuffledUnderSampledBalancedIndices